In [1]:
# 生成特征，思路
    #将所有数据合并成一张大表
    #包括：
        #资产负债表：GB_Balance_Sheet.xlsx
        #利润表：GB_Income_Statement.xlsx 
        #宏观经济指标表：MacroIndustry.xlsx
        #公司运营数据表：CompanyOperating.xlsx
        #股票行情数据表：MarketData.xlsx

    #没有包括：
        #现金流量表：GB_Cash_Flow_Statement.xlsx

# 需要调用的函数

In [2]:
def biningData(sr,n_Quantile,is_onehot):
     
    df_bins = pd.qcut(sr,n_Quantile,duplicates = 'drop').to_frame()
    
    df_bins['DIS_'+sr.name] = pd.Categorical(df_bins.iloc[:,0]).codes
    if(is_onehot==True):
        df_bins['DIS_'+sr.name]=df_bins['DIS_'+sr.name].astype('category')
    return(df_bins['DIS_'+sr.name])

In [3]:
def featureEngineeringTimeSeriesData(df_s,n_lag):
     
    list_col = df_s.columns[2:]
    df_append =  []
    for i in range(n_lag):
        #i = 0
        print(i+1)
        df_tmp = df_s.groupby(['TICKER_SYMBOL'])[list_col].shift(i+1)
        list_col_target = [col+'_'+'LAG'+str(i+1) for col in df_tmp.columns]
        df_tmp.columns = list_col_target
        df_append.append(df_tmp)
    df_s = pd.concat(df_append, axis=1)
    return(df_s)

In [4]:
def biningAndRankingData(df,n_Quantile,is_onehot):
    
    bin_columns = ['DIS_'+ i for i in df.columns]
    rank_columns = ['RANK_'+ i for i in df.columns]
     
    if(is_onehot==True):
        df_bins = df.apply(lambda x:biningData(x,n_Quantile=n_Quantile,is_onehot=is_onehot))
        df_bins.columns = bin_columns
        df_bins = pd.get_dummies(df_bins, prefix=bin_columns)
    else:
        df_bins = df.apply(lambda x:biningData(x,n_Quantile=n_Quantile,is_onehot=is_onehot))
        df_bins.columns = bin_columns
   

    df_bins[rank_columns] = df.rank(method = 'min')
    return(df_bins)

# 读取中间数据

In [5]:
#原函数的函数头
    # def createFeature(GB_Income_Statement,df_indus_g,df_comp_g,df_share_g,START_DATE=pd.Timestamp('2008-01-01'),
    #                   IS_ONEHOT=False,N_QUANTILES=100,N_LAG=5):

## 读取宏观经济数据

In [6]:
import pandas as pd
df_indus = pd.read_csv("Data/df_indus.csv",index_col=0)
    #原始数据为MacroIndustry.xlsx
    #已在tidyingMacroIndustry中进行规整化处理

## 读取公司运营数据

In [7]:
df_comp  = pd.read_csv("Data/df_comp.csv",index_col=0)
    #原始数据为公司运营数据表：CompanyOperating.xlsx
    #在tidyingCompanyOperating中进行规整化处理

## 读取股票行情数据

In [8]:
df_share = pd.read_csv("Data/df_share.csv" ,index_col=0)
    #元数据为股票行情数据表：MarketData.xlsx

## 读取用于特征生成的资产负债表

In [9]:
# df_bank_toCreateFeature.csv中已经把利润表放进来了
    #df_bank = fg.tidyBalanceAndIncomeData(df_BalanceSheet_Bank,df_IncomeStatement_Bank)
    
df_bank=pd.read_csv("Data/df_bank_toCreateFeature.csv", index_col= 0, converters={'END_DATE': pd.Timestamp})
    #注：df_bank必须df_bank_toCreateFeature.csv，该数据在主程序中产生

# 参数设置

In [10]:
DATETIME_UNITS = "QUARTER"#YEAR，QUARTER，HALF
START_DATE = pd.Timestamp('2008-01-01')#年份开始时间
IS_ONEHOT =  False#是否设成热独编码
N_QUANTILES = 80#离散份数：按多少分位数分
N_LAG = 5#偏移阶数
N_TREES = 100#因子筛选：树数量
P_FEATUREIMP = 0.9999#选择因子前多少百分比的重要性
P_FEATRUES = 0.8#随机森林特征选择比例

In [11]:
#传入的参数值
GB_Income_Statement = df_bank
df_indus_g = df_indus 
df_comp_g = df_comp 
df_share_g = df_share

In [12]:
"year" in df_bank.columns

True

# 将每个表中的'TICKER_SYMBOL'，year，month 三列的类型改为字符串

In [13]:
#一些列的取值准换为字符串
GB_Income_Statement['TICKER_SYMBOL'] = GB_Income_Statement['TICKER_SYMBOL'].astype(str)
df_comp_g['TICKER_SYMBOL'] = df_comp_g['TICKER_SYMBOL'].astype(str)
df_share_g['TICKER_SYMBOL'] = df_share_g['TICKER_SYMBOL'].astype(str)
GB_Income_Statement['year'] = GB_Income_Statement['year'].astype(str)
df_indus_g['year'] = df_indus_g['year'].astype(str)
df_comp_g['year'] = df_comp_g['year'].astype(str)
df_share_g['year'] = df_share_g['year'].astype(str)
GB_Income_Statement['month'] = GB_Income_Statement['month'].astype(str)
df_indus_g['month'] = df_indus_g['month'].astype(str)
df_comp_g['month'] = df_comp_g['month'].astype(str)
df_share_g['month'] = df_share_g['month'].astype(str)


# 合并 df_bank,df_comp,df_indus,df_share

In [14]:
print('*合并开始..............')
df_train = pd.merge(GB_Income_Statement,df_comp_g,how='left',on=['TICKER_SYMBOL','year', 'month']).fillna(0)
df_train = pd.merge(df_train,df_indus_g,how='left',on=['year', 'month']).fillna(0)
df_train = pd.merge(df_train,df_share_g,how='left',on=['TICKER_SYMBOL','year', 'month']).fillna(0)
   
print('*合并完毕..............')
df_train.shape

*合并开始..............
*合并完毕..............


(771, 738)

# 垃圾回收

In [15]:
# import gc
# del df_indus_g
# del df_comp_g
# del df_share_g
# del GB_Income_Statement
# gc.collect() #垃圾回收

# 时间过滤

In [16]:
df_train['END_DATE']

0     2008-12-31
1     2009-03-31
2     2009-06-30
3     2009-09-30
4     2009-12-31
         ...    
766   2017-03-31
767   2017-06-30
768   2017-09-30
769   2017-12-31
770   2018-03-31
Name: END_DATE, Length: 771, dtype: datetime64[ns]

In [17]:
#筛选时间
START_DATE = pd.Timestamp('2008-01-01')#年份开始时间
df_train = df_train[df_train['END_DATE']>=START_DATE]

In [18]:
df_train.shape

(768, 738)

# 删除方差过小的列

In [19]:
#排序
import numpy as np
import pandas as pd

df_train = df_train.drop(['year','month'],1).sort_values(['TICKER_SYMBOL','END_DATE'])
df_train.head()

,TICKER_SYMBOL,END_DATE,C_RESER_CB,DEPOS_IN_OTH_BFI,PRECI_METALS,LOAN_TO_OTH_BANK_FI,TRADING_FA,DERIV_ASSETS,PUR_RESALE_FA,INT_RECEIV,...,DATA_VALUE2170726263,DATA_VALUE2170726264,DATA_VALUE2170726265,DATA_VALUE2170726266,DATA_VALUE2210200588,Unnamed: 0.1,CLOSE_PRICE,TURNOVER_VOL,TURNOVER_VALUE,MARKET_VALUE
0,1,2008-12-31,3.976790e+10,2.150081e+10,9225000.0,9.236676e+09,4.144100e+07,290751000.0,3.473335e+10,1.605636e+09,...,6.460656,7.135,4.753279,1.349286,10.376667,7.0,3.019333,9.124747e+08,9.579367e+09,2.776258e+10
1,1,2009-03-31,3.954528e+10,3.281080e+10,7080000.0,9.836695e+09,1.363916e+09,200250000.0,3.247499e+10,1.946312e+09,...,6.460656,7.135,4.753279,1.349286,8.543333,8.0,4.658667,1.096579e+09,1.452570e+10,4.283428e+10
2,1,2009-06-30,4.222817e+10,2.081290e+10,5342000.0,7.998342e+09,1.785393e+09,162354000.0,3.795794e+10,1.711397e+09,...,6.460656,7.135,4.753279,1.349286,7.883333,9.0,6.302333,8.111175e+08,1.525179e+10,5.794739e+10
3,1,2009-09-30,4.542918e+10,1.781418e+10,4694000.0,8.416666e+09,1.408176e+09,173611000.0,3.602692e+10,1.660956e+09,...,6.460656,7.135,4.753279,1.349286,8.126667,10.0,7.248000,9.065716e+08,2.042128e+10,6.664261e+10
4,1,2009-12-31,5.424395e+10,1.559254e+10,3302000.0,5.361139e+09,1.132048e+09,99996000.0,4.092340e+10,1.625700e+09,...,6.460656,7.135,4.753279,1.349286,8.073333,11.0,8.006000,5.768548e+08,1.390735e+10,7.360913e+10


In [20]:
#替换
df_train = df_train.replace([np.inf, -np.inf], np.nan)

In [21]:
#向前填充
df_train = df_train.groupby("TICKER_SYMBOL").apply(lambda x:x.ffill())

In [22]:
#计算方差
sr_var = df_train.var(axis =0)

In [23]:
#选择方差过小的列名
kill_col = sr_var[sr_var<=0.0000001]
kill_col.size
#共有87个

87

In [24]:
#删除方差过小的列
df_train = df_train.drop(kill_col.index,1)

##### 测试
df_train.shape

(768, 649)

In [25]:
#读取训练集
df_train.to_csv('Data/df_train.csv')
df_train=pd.read_csv('Data/df_train.csv',index_col=0)


In [26]:
df_train.shape
#((768, 649), (768, 649))

(768, 649)

In [27]:
df_train.columns

Index(['TICKER_SYMBOL', 'END_DATE', 'C_RESER_CB', 'DEPOS_IN_OTH_BFI',
       'PRECI_METALS', 'LOAN_TO_OTH_BANK_FI', 'TRADING_FA', 'DERIV_ASSETS',
       'PUR_RESALE_FA', 'INT_RECEIV',
       ...
       'DATA_VALUE2170726263', 'DATA_VALUE2170726264', 'DATA_VALUE2170726265',
       'DATA_VALUE2170726266', 'DATA_VALUE2210200588', 'Unnamed: 0.1',
       'CLOSE_PRICE', 'TURNOVER_VOL', 'TURNOVER_VALUE', 'MARKET_VALUE'],
      dtype='object', length=649)

# 产生窗口特征

In [28]:
df_train_lag = featureEngineeringTimeSeriesData(df_train,N_LAG)
df_train_f =  pd.concat([df_train,df_train_lag],axis=1)

1
2
3
4
5


# 特征的离散化处理

In [29]:
df_dis_rank = biningAndRankingData(df_train.iloc[:,2:],n_Quantile=N_QUANTILES,is_onehot=IS_ONEHOT)#排序
df_train_f =  pd.concat([df_train_f,df_dis_rank],axis=1)
df_train_f = df_train_f.fillna(0)
df_train_f.head()

调用biningAndRankingData（），开始进行分箱和赋排名处理-------


,TICKER_SYMBOL,END_DATE,C_RESER_CB,DEPOS_IN_OTH_BFI,PRECI_METALS,LOAN_TO_OTH_BANK_FI,TRADING_FA,DERIV_ASSETS,PUR_RESALE_FA,INT_RECEIV,...,RANK_DATA_VALUE2170726263,RANK_DATA_VALUE2170726264,RANK_DATA_VALUE2170726265,RANK_DATA_VALUE2170726266,RANK_DATA_VALUE2210200588,RANK_Unnamed: 0.1,RANK_CLOSE_PRICE,RANK_TURNOVER_VOL,RANK_TURNOVER_VALUE,RANK_MARKET_VALUE
0,1,2008-12-31,3.976790e+10,2.150081e+10,9225000.0,9.236676e+09,4.144100e+07,290751000.0,3.473335e+10,1.605636e+09,...,190.0,165.0,136.0,162.0,712.0,97.0,208.0,390.0,449.0,160.0
1,1,2009-03-31,3.954528e+10,3.281080e+10,7080000.0,9.836695e+09,1.363916e+09,200250000.0,3.247499e+10,1.946312e+09,...,190.0,165.0,136.0,162.0,287.0,98.0,386.0,441.0,559.0,202.0
2,1,2009-06-30,4.222817e+10,2.081290e+10,5342000.0,7.998342e+09,1.785393e+09,162354000.0,3.795794e+10,1.711397e+09,...,190.0,165.0,136.0,162.0,216.0,99.0,509.0,360.0,572.0,231.0
3,1,2009-09-30,4.542918e+10,1.781418e+10,4694000.0,8.416666e+09,1.408176e+09,173611000.0,3.602692e+10,1.660956e+09,...,190.0,165.0,136.0,162.0,253.0,100.0,558.0,385.0,633.0,255.0
4,1,2009-12-31,5.424395e+10,1.559254e+10,3302000.0,5.361139e+09,1.132048e+09,99996000.0,4.092340e+10,1.625700e+09,...,190.0,165.0,136.0,162.0,233.0,101.0,594.0,267.0,551.0,280.0
